<a href="https://colab.research.google.com/github/itchyfeet-patient/Beautiful-Exploration/blob/master/Exploration_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **13-9. 프로젝트 - Movielens 영화 추천 실습** 🎥

이전 스텝에서 배운 MF 모델 학습 방법을 토대로, 내가 좋아할 만한 영화 추천 시스템을 제작해 보겠습니다.

이번에 활용할 데이터셋은 추천 시스템의 MNIST라고 부를만한 Movielens 데이터입니다.

* 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
* 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해 볼 수 있습니다.
* 별점을 시청횟수로 해석해서 생각하겠습니다.
* 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.  

Cloud Storage에 미리 업로드된 ml-1m폴더 내 파일을 심볼릭 링크로 개인 storage에 연결해 줍니다.

Cloud shell에서 아래 명령어를 입력해 주세요.
```
$ mkdir -p ~/aiffel/recommendata_iu/data/ml-1m
$ ln -s ~/data/ml-1m/* ~/aiffel/recommendata_iu/data/ml-1m
```
프로젝트에 필요한 주요 라이브러리 버전을 확인해봅시다.

In [ ]:
!pip install 'scipy'=='1.7.1' --force-reinstall

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached scipy-1.7.1-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (28.5 MB)
  Using cached numpy-1.21.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.3
    Uninstalling scipy-1.7.3:
      Successfully uninstalled scipy-1.7.3


In [ ]:
pip install --upgrade pip setuptools wheel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached pip-22.2.2-py3-none-any.whl (2.0 MB)
  Using cached setuptools-65.3.0-py3-none-any.whl (1.2 MB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.


In [ ]:
!pip install 'implicit'=='0.4.8' --force-reinstall

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached implicit-0.4.8.tar.gz (1.1 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached numpy-1.21.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
  Using cached scipy-1.7.3-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (38.1 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 3.8 MB/s eta 0:00:00
  Created wheel for implicit: filename=implicit-0.4.8-cp37-cp37m-linux_x86_64.whl size=4620669 sha256=bd327e878b95495601b7e5cef93305a9a3cc7a47f3974c39c523eb26bfe62aba
  Stored in directory: /root/.cache/pip/wheels/88/e6/34/25e73cccbaf1a961154bb562a5f86123b68fdbf40e306073d6
Successfully built implicit
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.64.0
    Uninstalling tqdm-4.64.0:


In [ ]:
import numpy as np
import scipy
import implicit
import pandas as pd

print(np.__version__)
print(scipy.__version__)
print(implicit.__version__)

# lms 버전
# 1.21.4
# 1.7.1
# 0.4.8

1.21.6
1.7.3
0.4.8


## **1) 데이터 준비와 전처리**
Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.



In [ ]:
import os
rating_file_path= '/content/drive/MyDrive/dataset/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정했으므로 3점 이상의 데이터만 남겨줍니다.

In [ ]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [ ]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [ ]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [ ]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path= '/content/drive/MyDrive/dataset/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


movie_id가 어떤 영화를 나타내는 지 알 수 있습니다. 

여기까지가 전처리입니다. 이후에는 이전 스텝에 소개했던 것과 동일한 방식으로 MF model을 구성하여 내가 좋아할 만한 영화를 추천해 볼 수 있습니다.

In [ ]:
movies['title'] = movies['title'].str.lower() # 검색을 쉽게 하기 위해 아티스트 문자열을 소문자로 바꿔줍시다.
movies['genre'] = movies['genre'].str.lower()

# genre 전처리 추가하기
movies.head(10)

,movie_id,title,genre
0,1,toy story (1995),animation|children's|comedy
1,2,jumanji (1995),adventure|children's|fantasy
2,3,grumpier old men (1995),comedy|romance
3,4,waiting to exhale (1995),comedy|drama
4,5,father of the bride part ii (1995),comedy
5,6,heat (1995),action|crime|thriller
6,7,sabrina (1995),comedy|romance
7,8,tom and huck (1995),adventure|children's
8,9,sudden death (1995),action
9,10,goldeneye (1995),action|adventure|thriller


In [ ]:
df = pd.merge(movies, ratings, on='movie_id')
df

,movie_id,title,genre,user_id,counts,timestamp
0,1,toy story (1995),animation|children's|comedy,1,5,978824268
1,1,toy story (1995),animation|children's|comedy,6,4,978237008
2,1,toy story (1995),animation|children's|comedy,8,4,978233496
3,1,toy story (1995),animation|children's|comedy,9,5,978225952
4,1,toy story (1995),animation|children's|comedy,10,5,978226474
...,...,...,...,...,...,...
836473,3952,"contender, the (2000)",drama|thriller,5682,3,1029457829
836474,3952,"contender, the (2000)",drama|thriller,5812,4,992072099
836475,3952,"contender, the (2000)",drama|thriller,5831,3,986223125
836476,3952,"contender, the (2000)",drama|thriller,5837,4,1011902656


In [ ]:
using_cols = ['user_id', 'title', 'counts']
df = df[using_cols]
df.head(10)

,user_id,title,counts
0,1,toy story (1995),5
1,6,toy story (1995),4
2,8,toy story (1995),4
3,9,toy story (1995),5
4,10,toy story (1995),5
5,18,toy story (1995),4
6,19,toy story (1995),5
7,21,toy story (1995),3
8,23,toy story (1995),4
9,26,toy story (1995),3


## **2) 분석해 봅시다.**
* ratings에 있는 유니크한 영화 개수
* ratings에 있는 유니크한 사용자 수
* 가장 인기 있는 영화 30개(인기순)

일단 첫번째 유저가 어떻게 평점을 줬는지 한번 보겠습니다.

In [ ]:
condition = (df['user_id']== ratings.loc[0, 'user_id'])
df.loc[condition]


,user_id,title,counts
0,1,toy story (1995),5
19773,1,pocahontas (1995),5
35123,1,apollo 13 (1995),5
54895,1,star wars: episode iv - a new hope (1977),4
116019,1,schindler's list (1993),5
119008,1,"secret garden, the (1993)",4
130190,1,aladdin (1992),4
138999,1,snow white and the seven dwarfs (1937),4
139706,1,beauty and the beast (1991),5
142605,1,fargo (1996),4


'뻐꾸기 둥지 위로 날아간 새' 영화에 5점을 줬네요~ 전체적으로 점수가 후합니다.

### ratings에 있는 유니크한 영화 개수

In [ ]:
df['title'].nunique()

3628

### ratings에 있는 유니크한 사용자 수

In [ ]:
df['user_id'].nunique()

6039

### 가장 인기 있는 영화 30개(인기순)

In [ ]:
artist_count = df.groupby('title')['counts'].sum()
artist_count_result = artist_count.sort_values(ascending=False).head(30)
artist_count_result=pd.DataFrame(artist_count_result)
artist_count_result

,counts
title,
american beauty (1999),14449
star wars: episode iv - a new hope (1977),13178
star wars: episode v - the empire strikes back (1980),12648
saving private ryan (1998),11348
star wars: episode vi - return of the jedi (1983),11303
raiders of the lost ark (1981),11179
"silence of the lambs, the (1991)",11096
"matrix, the (1999)",10903
"sixth sense, the (1999)",10703


rating(counts)을 sum 한 값이라서 counts 란에 해당 영화를 본 횟수가 sum되어있습니다.

아메리칸 뷰티, 스타워즈, 라이언 일병 구하기 등 익숙한 고전 명작 영화가 순위권에 즐비하네요

유저별 몇 번의 영화를 봤는지 보겠습니다.

In [ ]:
user_count = df.groupby('user_id')['counts'].sum()
user_count.describe()

count    6039.000000
mean      548.273721
std       606.449800
min         3.000000
25%       154.000000
50%       329.000000
75%       704.000000
max      7590.000000
Name: counts, dtype: float64

In [ ]:
user_count

user_id
1        222
2        455
3        190
4         85
5        532
        ... 
6036    2620
6037     727
6038      73
6039     469
6040    1117
Name: counts, Length: 6039, dtype: int64

가장 많은 영화를 본 사람은 7590 회까지 봤네요~ 보통 548회정도 봤다고 합니다 대단해

너무 많은 것 같기도 한데 이건 별점을 시청횟수로 간주했기 때문에 벌어진 일입니다..

## **3) 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.**

In [ ]:
# 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰줘야한다!
my_favorite = ['chicken run (2000)' , 'shawshank redemption, the (1994)' ,'pulp fiction (1994)' ,"bug's life, a (1998)" ,'erin brockovich (2000)']

# 'jumi'이라는 user_id가 위 영화를 시청한 횟수(별점)가 5씩이라고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['jumi']*5, 'title': my_favorite, 'counts':[5]*5})

if not ratings.isin({'user_id':['jumi']})['user_id'].any():  # user_id에 'jumi'이라는 데이터가 없다면
    df = df.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

df.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,title,counts
836473,5682,"contender, the (2000)",3
836474,5812,"contender, the (2000)",4
836475,5831,"contender, the (2000)",3
836476,5837,"contender, the (2000)",4
836477,5998,"contender, the (2000)",4
0,jumi,chicken run (2000),5
1,jumi,"shawshank redemption, the (1994)",5
2,jumi,pulp fiction (1994),5
3,jumi,"bug's life, a (1998)",5
4,jumi,erin brockovich (2000),5


잘 추가되었네요~

In [ ]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = df['user_id'].unique()
movie_id_unique = df['title'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_id_to_idx = {v:k for k,v in enumerate(movie_id_unique)}

In [ ]:
df['user_id'].nunique()

6040

In [ ]:
print(user_to_idx['jumi'])    # 6040명의 유저 중 마지막으로 추가된 유저이니 6039가 나와야 합니다. 


6039


In [ ]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = df['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(df):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    df['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_id_data = df['title'].map(movie_id_to_idx.get).dropna()
if len(temp_movie_id_data) == len(df):
    print('title column indexing OK!!')
    df['title'] = temp_movie_id_data
else:
    print('title column indexing Fail!!')

df

user_id column indexing OK!!
title column indexing OK!!


,user_id,title,counts
0,0,0,5
1,1,0,4
2,2,0,4
3,3,0,5
4,4,0,5
...,...,...,...
0,6039,3433,5
1,6039,305,5
2,6039,283,5
3,6039,2114,5


## **4) CSR matrix를 직접 만들어 봅시다.**

In [ ]:
from scipy.sparse import csr_matrix

num_user = df['user_id'].nunique()
num_movie = df['title'].nunique()

csr_data = csr_matrix((df.counts, (df.user_id, df.title)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## **5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.**


In [ ]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [ ]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=300, regularization=0.01, use_gpu=False, iterations=100, dtype=np.float32)

In [ ]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [ ]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/100 [00:00<?, ?it/s]

## **6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.**

너무너무 재밌는 애니메이션 치킨런입니다.. 감동 🐔 🐤

In [ ]:
jumi, chicken_run = user_to_idx['jumi'], movie_id_to_idx['chicken run (2000)']
jumi_vector, chicken_run_vector = als_model.user_factors[jumi], als_model.item_factors[chicken_run]

print('슝=3')

슝=3


In [ ]:
jumi_vector

array([ 1.44828871e-01,  1.23439506e-01, -2.80297279e-01,  2.39950448e-01,
       -3.92419249e-01, -3.69699001e-01, -1.90648958e-02,  5.73755167e-02,
       -8.99208859e-02, -4.28472459e-02,  9.41368788e-02,  4.69745062e-02,
        3.73590499e-01,  2.63949782e-01, -1.40150547e-01, -4.13345367e-01,
       -2.21703306e-01,  1.86629385e-01, -2.92054951e-01, -2.56979108e-01,
        5.61946705e-02, -1.96970299e-01, -6.39449954e-02,  1.48100302e-01,
        1.13556072e-01, -9.67146233e-02, -5.52164279e-02, -1.58128008e-01,
        1.43814147e-01, -2.28259757e-01,  7.54811838e-02,  2.40348756e-01,
       -4.44228053e-02,  2.85815954e-01, -6.98581757e-03, -4.41561751e-02,
        1.38067394e-01,  2.51470000e-01, -4.20246385e-02, -3.45215313e-02,
        2.61311531e-01, -4.37466577e-02, -2.49598190e-01,  1.28917888e-01,
       -2.53840774e-01,  1.19846247e-01,  3.20963204e-01,  9.60411578e-02,
       -3.94566059e-02, -1.03526842e-02,  1.21760508e-02, -2.14722976e-01,
       -1.56794950e-01,  

In [ ]:
chicken_run_vector

array([ 3.27575691e-02,  1.41779091e-02,  7.67479884e-03,  3.35771367e-02,
       -3.18204649e-02, -3.89600173e-02,  4.89066616e-02,  5.35822287e-03,
       -5.90545498e-02, -2.53488310e-04, -3.81781757e-02, -4.04532393e-03,
        5.94547652e-02,  3.94766480e-02, -2.22946629e-02,  1.60892867e-02,
       -1.34735014e-02,  1.44637367e-02, -3.09066586e-02, -7.30219260e-02,
        6.44242018e-03, -1.26195047e-02, -1.75346751e-02,  9.51057673e-03,
        4.58699726e-02, -4.76089828e-02,  1.40704690e-02, -4.41891290e-02,
        9.85765923e-03,  2.09001489e-02, -5.53288357e-03, -2.79018495e-05,
        1.16414949e-02, -2.05404498e-02,  2.27376744e-02, -6.87794760e-02,
        1.63687374e-02,  5.62020428e-02, -7.92438816e-03, -2.27791090e-02,
        1.69931129e-02,  2.94721629e-02, -2.22893152e-02,  6.72089085e-02,
       -1.09458193e-02, -2.52839681e-02,  2.94379164e-02, -2.74646818e-03,
       -5.76896407e-02, -7.16549009e-02, -3.97881530e-02, -7.77243404e-03,
       -1.28800105e-02,  

In [ ]:
np.dot(jumi_vector, chicken_run_vector)

0.84039736

1에 가까우면 좋겠는데 그래도 0.6이면 나쁘지않네요, factor를 늘리거나 iteration을 늘려야 할 것 같답니다?  
-> factor를 500, iteration을 30으로 했더니 1에 가까운 0.9153 값이 나왔네요!

In [ ]:
toy_story = movie_id_to_idx['toy story (1995)']
toy_vector = als_model.item_factors[toy_story]
np.dot(jumi_vector, toy_vector)

0.095197596

In [ ]:
s_list = movie_id_to_idx["schindler's list (1993)"]
s_list_vector = als_model.item_factors[s_list]
np.dot(jumi_vector, s_list_vector)

0.09148958

## **7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.**

AlternatingLeastSquares 클래스에 구현되어 있는 `similar_items` 메서드를 통하여 비슷한 영화를 찾습니다. 아까도 사용했던 치킨런으로 비슷한 영화를 추천받아보겠습니다.

In [ ]:
favorite_movie = 'chicken run (2000)'
artist_id = movie_id_to_idx[favorite_movie]
similar_movie = als_model.similar_items(artist_id, N=15)
similar_movie

[(3433, 1.0),
 (3307, 0.25984147),
 (3436, 0.25901368),
 (693, 0.24769531),
 (1107, 0.24218953),
 (1300, 0.23908626),
 (3130, 0.23876856),
 (3428, 0.23227207),
 (3182, 0.22387223),
 (3262, 0.22027485),
 (3510, 0.21412477),
 (3304, 0.20749384),
 (3430, 0.20607232),
 (3252, 0.20495552),
 (2372, 0.20366952)]

(영화id, 유사도) 형태의 tuple로 반환하고 있습니다. 영화 id를 다시 영화 이름으로 매핑시켜줍니다.

In [ ]:
# movie_id_to_idx 를 뒤집어, index로부터 movie 이름을 얻는 dict를 생성합니다. 
idx_to_movie_id = {v:k for k,v in movie_id_to_idx.items()}
[idx_to_movie_id[i[0]] for i in similar_movie]

['chicken run (2000)',
 'small time crooks (2000)',
 'adventures of rocky and bullwinkle, the (2000)',
 'close shave, a (1995)',
 'grand day out, a (1992)',
 'machine, the (1994)',
 'creature comforts (1990)',
 'titan a.e. (2000)',
 'road to el dorado, the (2000)',
 'idiots, the (idioterne) (1998)',
 'saving grace (2000)',
 'dinosaur (2000)',
 "jesus' son (1999)",
 'virgin suicides, the (1999)',
 "trippin' (1999)"]

In [ ]:
def get_similar_movie(movie_name: str):
    movie_id = movie_id_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie_id[i[0]] for i in similar_movie]
    return similar_movie

print("슝=3")

슝=3


In [ ]:
get_similar_movie('shawshank redemption, the (1994)')

['shawshank redemption, the (1994)',
 'good will hunting (1997)',
 'rage: carrie 2, the (1999)',
 'silence of the lambs, the (1991)',
 'goodfellas (1990)',
 'pulp fiction (1994)',
 'jerry & tom (1998)',
 'ulysses (ulisse) (1954)',
 'innocents, the (1961)',
 "schindler's list (1993)"]

실화기반, 드라마 장르의 영화들이 꽤 나왔습니다. 비슷한 것 같습니다.

In [ ]:
get_similar_movie('pulp fiction (1994)')

['pulp fiction (1994)',
 'goodfellas (1990)',
 'reservoir dogs (1992)',
 'fargo (1996)',
 'back stage (2000)',
 'jackie brown (1997)',
 'shawshank redemption, the (1994)',
 'usual suspects, the (1995)',
 'condition red (1995)',
 'good will hunting (1997)']

좋은 친구들, 저수지의 개들 같은 매우 유사한 장르의 영화가 많이 나왔네요. 저수지의 개들은 펄프픽션의 감독과 같은 쿠엔틴 타란티노 감독의 작품으로, 아주 비슷한 영화죠!

## **8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.**

AlternatingLeastSquares 클래스에 구현되어 있는 `recommend` 메서드를 통하여 제가 좋아할 만한 영화를 추천받습니다. filter_already_liked_items 는 유저가 이미 평가한 아이템은 제외하는 Argument입니다.

In [ ]:
user = user_to_idx['jumi']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(587, 0.24533886),
 (2845, 0.2412419),
 (1097, 0.20556715),
 (2878, 0.1849227),
 (2155, 0.17297412),
 (1109, 0.1513806),
 (2056, 0.14446332),
 (3231, 0.14146689),
 (1547, 0.13779077),
 (1532, 0.13693616),
 (3270, 0.13301389),
 (573, 0.12851314),
 (3304, 0.12807454),
 (2877, 0.12622863),
 (2470, 0.12322686),
 (3180, 0.122345924),
 (2968, 0.122273766),
 (995, 0.11962016),
 (35, 0.11889484),
 (2406, 0.11657454)]

In [ ]:
[idx_to_movie_id[i[0]] for i in movie_recommended]

['fargo (1996)',
 'toy story 2 (1999)',
 'goodfellas (1990)',
 'cider house rules, the (1999)',
 'shakespeare in love (1998)',
 'amadeus (1984)',
 'antz (1998)',
 '28 days (2000)',
 'jackie brown (1997)',
 'good will hunting (1997)',
 'gladiator (2000)',
 'silence of the lambs, the (1991)',
 'dinosaur (2000)',
 'green mile, the (1999)',
 'runaway bride (1999)',
 'high fidelity (2000)',
 "wayne's world (1992)",
 'reservoir dogs (1992)',
 'dead man walking (1995)',
 'rocky horror picture show, the (1975)']

좋은 친구들을 추천해주네요! 왠지 펄프픽션 때문인 것 같습니다.

In [ ]:
toy_story = movie_id_to_idx['goodfellas (1990)']
explain = als_model.explain(user, csr_data, itemid=toy_story)

In [ ]:
[(idx_to_movie_id[i[0]], i[1]) for i in explain[1]]

[('pulp fiction (1994)', 0.1770970355960667),
 ('shawshank redemption, the (1994)', 0.06608828557387775),
 ("bug's life, a (1998)", 0.005277838169644559),
 ('erin brockovich (2000)', -0.015549937634761927),
 ('chicken run (2000)', -0.028206468913068435)]

음.. 기여도가 전체적으로 낮긴 한데 제 예상처럼 펄프픽션이 가장 기여도 비중이 높습니다. 펄프픽션때문에 추천해준듯

## 🌮 루브릭
| **평가문항** | **상세기준** | **성취결과** |
|:---:|---|:---:|
| 1. CSR matrix가 정상적으로 만들어졌다. | 사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다. | O |
| 2. MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다. | 사용자와 아이템 벡터 내적수치가 의미있게 형성되었다. | O |
| 3. 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다. | MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도를 측정하고 의미를 분석해보았다. | O |

## ✍ 회고

* csr 행렬 생성에서 오류가 생겼다. 인덱스가 뭐 범위밖이라나? 세영님이 버전 문제일거라고 하셔서 lms 버전이랑 일치시켜줬다. thx 🙏 (implicit)   
그런데 `Could not build wheels for implicit which use PEP 517 and cannot be installed directly` 이라는 오류가 또 생겨서 검색해보니   
`pip install --upgrade pip setuptools wheel` 한줄 실행해주면 된다고 해서 그대로 했다.  
성공!



* 나의 선호 영화와 나의 csr 벡터를 내적했을 때 1에 가까이 나와야 한다고 했는데, 초반에 0.6? 정도가 나와서 iteration과 factor를 두배로 늘려줬다. 그 결과 chicken run은 0.9에 육박하는 결과가 나왔지만,, 쇼생크 탈출과 펄프픽션 같은 영화는 0에 가까운 값이 나왔다. 과적합된것일까?  
    * factor, iteration을 적절하게 조절해 본 결과, 적절한 평형을 유지하는 그 지점을 찾는게 좋아보인다. 지금 결과값으로는 치킨런이 0.8정도고, 토이스토리와 쉰들러리스트는 0.1이 나온다. 이전 시행에서 0.9와 0.01이 나온 것보다는 낫다고 생각.

* 마지막에 내가 좋아할 것 같은 영화들을 추천해줬는데, 최신 DB로 했으면 더 재밌고 공감됐을 것 같다는 생각이 들었다. 그렇지만 현재 데이터로 만든 결과도 꽤 정확했다. 어떤 영화때문에 추천해 준지 알 것 같았고, 평점을 시청시간으로 간주해서 평점이 높은 영화들 위주로 추천해준게 보였다.  
다만 평점을 시청시간으로 간주해서 좀 부정확하게 느껴졌고 정확한 시청시간을 안다거나 암묵적인 평가를 더 추가해서 csr 행렬을 만든다면 더욱 성능 좋은 추천 시스템을 만들수 있겠다고 생각이 되었다.

* 펄프 픽션, 치킨런, 쇼생크탈출.. 같은 내 영화 variation이 넓기도 해서 부정확한 결과가 좀 나온듯 하다. 액션/애니메이션/실화기반 드라마? 가 한데 모임. 그런데 이런 잡식성 취향도 잘 분석을 해서 추천해줘야 진정한 추천 시스템이 아닐까?!!!

* 그리고 프로젝트 전 사전 실습에서 spotify 데이터셋을 이용해서 추천했는데, 너무 재밌었다! spotify 를 쓰고 있기도 하고, 2010년도에도 외국노래를 나름 활발하게 들었던 사람으로서,,, 반가운 이름들도 많이 보였고 재밌었다. 역시 내 생활과 밀접한 관련이 있는 내용은 더 흥미롭다.

### 📚 참고문헌 

[패키지 설치 시 PEP 오류 해결 방법](https://heehehe-ds.tistory.com/entry/Python-%ED%8C%A8%ED%82%A4%EC%A7%80-%EC%84%A4%EC%B9%98-%EC%8B%9C-PEP-%EC%98%A4%EB%A5%98-%ED%95%B4%EA%B2%B0-%EB%B0%A9%EB%B2%95)